<img src="https://github.com/hernancontigiani/ceia_memorias_especializacion/raw/master/Figures/logoFIUBA.jpg" width="500" align="center">


# Procesamiento de lenguaje natural
## Word Embeddings con Glove y Fasttext

Este código permite levantar los embeddings de fastText y de GloVe y poder utilizarlos para calcular la distancia entre palabras

In [3]:
import logging

import os
from pathlib import Path
from io import StringIO
import pickle

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import gdown

### Datos

In [2]:
url = 'https://drive.google.com/u/0/uc?id=1Qi1r-u5lsEsNqRSxLrpNOqQ3B_ufltCa'
output = 'fasttext.pkl'
gdown.download(url, output, quiet=False)

Downloading...
From (original): https://drive.google.com/u/0/uc?id=1Qi1r-u5lsEsNqRSxLrpNOqQ3B_ufltCa
From (redirected): https://drive.google.com/uc?id=1Qi1r-u5lsEsNqRSxLrpNOqQ3B_ufltCa&confirm=t&uuid=072ca5ed-9a89-47c3-9268-fdd2385909a6
To: /Users/alejandrolloveras/Documents/ESTUDIO/UBA/Materias/NLP/Repositorio/clase_2/jupyter_notebooks/fasttext.pkl
 13%|█▎        | 370M/2.88G [00:12<01:13, 34.2MB/s] 

KeyboardInterrupt: 

In [ ]:
# Descargar los embeddings desde un gogle drive (es la forma más rápida)
# NOTA: No hay garantía de que estos links perduren, en caso de que no estén
# disponibles descargar de la página oficial como se explica en el siguiente bloque
# Download fastText Embeddings 
#!curl -L -o 'fasttext.pkl' 'https://drive.google.com/u/0/uc?id=1Qi1r-u5lsEsNqRSxLrpNOqQ3B_ufltCa&export=download&confirm=t'

# Download GloVe Embeddings
#!curl -L -o 'gloveembedding.pkl' 'https://drive.google.com/u/0/uc?id=1KY6avD5I1eI2dxQzMkR3WExwKwRq2g94&export=download&confirm=t'

In [ ]:
# De donde se obtuvieron estos embeddings:

# Link de donde se dscargo los embeddings de GloVe (de ese zip solo se sube el embedding de 50d)
# http://nlp.stanford.edu/data/glove.twitter.27B.zip
# Link de descarga de los embeddings 300 de fastText
# https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.en.300.vec.gz

In [ ]:
# Paper de GloVe
# https://nlp.stanford.edu/pubs/glove.pdf

# Papers de FastText
# https://arxiv.org/pdf/1607.04606.pdf
# https://arxiv.org/abs/1607.01759
# https://arxiv.org/abs/1612.03651

### 1 - Ensayar los embeddings de Glove y Fasttext

In [ ]:
# Acá definimos una clase que nos va a permitir manejar los embeddings de GloVe
# y fastText con la misma interfase.
# Incluye cómo cargar los embeddings a partir de sus pickles 
# (formato de datos serializados de Python) y guardarlos
# funciones para obtener términos dados índices y viceversa

class WordsEmbeddings(object):
    logger = logging.getLogger(__name__)

    def __init__(self):
        # load the embeddings
        words_embedding_pkl = Path(self.PKL_PATH)
        if not words_embedding_pkl.is_file():
            words_embedding_txt = Path(self.WORD_TO_VEC_MODEL_TXT_PATH)
            assert words_embedding_txt.is_file(), 'Words embedding not available'
            embeddings = self.convert_model_to_pickle()
        else:
            embeddings = self.load_model_from_pickle()
        self.embeddings = embeddings
        # build the vocabulary hashmap
        index = np.arange(self.embeddings.shape[0])
        # Dicctionarios para traducir de embedding a IDX de la palabra
        self.word2idx = dict(zip(self.embeddings['word'], index))
        self.idx2word = dict(zip(index, self.embeddings['word']))

    def get_words_embeddings(self, words):
        words_idxs = self.words2idxs(words)
        return self.embeddings[words_idxs]['embedding']

    def words2idxs(self, words):
        return np.array([self.word2idx.get(word, -1) for word in words])

    def idxs2words(self, idxs):
        return np.array([self.idx2word.get(idx, '-1') for idx in idxs])

    def load_model_from_pickle(self):
        self.logger.debug(
            'loading words embeddings from pickle {}'.format(
                self.PKL_PATH
            )
        )
        max_bytes = 2**28 - 1 # 256MB
        bytes_in = bytearray(0)
        input_size = os.path.getsize(self.PKL_PATH)
        with open(self.PKL_PATH, 'rb') as f_in:
            for _ in range(0, input_size, max_bytes):
                bytes_in += f_in.read(max_bytes)
        embeddings = pickle.loads(bytes_in)
        self.logger.debug('words embeddings loaded')
        return embeddings

    def convert_model_to_pickle(self):
        # create a numpy strctured array:
        # word     embedding
        # U50      np.float32[]
        # word_1   a, b, c
        # word_2   d, e, f
        # ...
        # word_n   g, h, i
        self.logger.debug(
            'converting and loading words embeddings from text file {}'.format(
                self.WORD_TO_VEC_MODEL_TXT_PATH
            )
        )
        structure = [('word', np.dtype('U' + str(self.WORD_MAX_SIZE))),
                     ('embedding', np.float32, (self.N_FEATURES,))]
        structure = np.dtype(structure)
        # load numpy array from disk using a generator
        with open(self.WORD_TO_VEC_MODEL_TXT_PATH, encoding="utf8") as words_embeddings_txt:
            embeddings_gen = (
                (line.split()[0], line.split()[1:]) for line in words_embeddings_txt
                if len(line.split()[1:]) == self.N_FEATURES
            )
            embeddings = np.fromiter(embeddings_gen, structure)
        # add a null embedding
        null_embedding = np.array(
            [('null_embedding', np.zeros((self.N_FEATURES,), dtype=np.float32))],
            dtype=structure
        )
        embeddings = np.concatenate([embeddings, null_embedding])
        # dump numpy array to disk using pickle
        max_bytes = 2**28 - 1 # # 256MB
        bytes_out = pickle.dumps(embeddings, protocol=pickle.HIGHEST_PROTOCOL)
        with open(self.PKL_PATH, 'wb') as f_out:
            for idx in range(0, len(bytes_out), max_bytes):
                f_out.write(bytes_out[idx:idx+max_bytes])
        self.logger.debug('words embeddings loaded')
        return embeddings

# Armamos clases particulares para manejar los embeddings de Glove y Fasttext
# que heredan de la clase anterior WordsEmbeddings
class GloveEmbeddings(WordsEmbeddings):
    WORD_TO_VEC_MODEL_TXT_PATH = 'glove.twitter.27B.50d.txt'
    PKL_PATH = 'gloveembedding.pkl'
    N_FEATURES = 50
    WORD_MAX_SIZE = 60


class FasttextEmbeddings(WordsEmbeddings):
    WORD_TO_VEC_MODEL_TXT_PATH = 'cc.en.300.vec'
    PKL_PATH = 'fasttext.pkl'
    N_FEATURES = 300
    WORD_MAX_SIZE = 60

In [ ]:
# Armamos una función para graficar la matriz de similaridad
def plot_matrix_distance(words, dist):
    fig = plt.figure(figsize=(16,9))
    ax = fig.add_subplot()
    sns.heatmap(dist, xticklabels=words, yticklabels=words, 
                annot=True, fmt=".2f", cmap="YlGnBu", ax=ax, mask=np.triu(dist))
    plt.show()

In [ ]:
# Ojo que si usan scipy la distancia coseno no es la similitud coseno (esta "negada")
from sklearn.metrics import pairwise

def embeddings_matrix_distance(model, words):
    print("Cantidad de palabras:", len(words))
    emb = model.get_words_embeddings(words)
    print("Dimensiones de los embeddings:", emb.shape)
    dist = pairwise.cosine_similarity(emb, emb)
    plot_matrix_distance(words, dist)

 13%|█▎        | 373M/2.88G [00:30<01:13, 34.2MB/s]

In [ ]:
# Instanciamos embedding de fasttext
# Puede demorar unos segundos porque tiene que cargar en memoria los archivos de embeddings
model_fasttext = FasttextEmbeddings()
words_fasttext=[model_fasttext.embeddings[i][0] for i in range(model_fasttext.embeddings.shape[0])]

In [ ]:
# cantidad de palabras en fasttext
model_fasttext.embeddings.shape[0]

In [ ]:
# Palabras a ensayar
# Algunas relativas con saludos y otras con dispositivos
words = ["hi", "hello", "bye", "goodbye", "morning", "computer", "machine", "laptop", "device", "printer"]

In [ ]:
embeddings_matrix_distance(model_fasttext, words)

In [ ]:
# probamos un test de analogía
test_words = ['king', 'man', 'woman', 'queen']
test_emb = model_fasttext.get_words_embeddings(test_words)

# king - man + woman
new_queen = test_emb[0] - test_emb[1] + test_emb[2]

new_words = test_words + ['new_queen']
new_emb = np.append(test_emb, new_queen.reshape(1, -1), axis=0)

dist = pairwise.cosine_similarity(new_emb, new_emb)

plot_matrix_distance(new_words, dist)

In [ ]:
# mas allá de las palabras propuestas, podemos ver cuál sería la palabra 
# cuyo embedding es más similar al de 'new_queen'
# para ello vamos a disponibilizar los embeddings como un numpy array
embeddings_fasttext = []
for idx,emb in enumerate(model_fasttext.embeddings):
  embeddings_fasttext.append(emb[1])
embeddings_fasttext = np.array(embeddings_fasttext)

In [ ]:
# calculamos la similaridad de 'new_queen' contra todos los vectores
# puede tardar unos segundos
similarities = np.squeeze(pairwise.cosine_similarity(embeddings_fasttext,np.array([new_queen])))

In [ ]:
# vemos cuáles son los 10 vectores más similares
args_sorted = np.argsort(similarities)[::-1][:10]
for arg in args_sorted:
  print(f'{words_fasttext[arg]}: {similarities[arg]}')

In [ ]:
test_words2 = ['anarchy', 'monarchy', 'kingdom', 'crown', 'royal']
test_emb2 = model_fasttext.get_words_embeddings(test_words2)

# king - man
no_king = test_emb[0] - test_emb[1]

new_words2 = test_words2 + ['no_king']
new_emb2 = np.append(test_emb2, no_king.reshape(1, -1), axis=0)

dist2 = pairwise.cosine_similarity(new_emb2, new_emb2)

plot_matrix_distance(new_words2, dist2)

In [ ]:
# king - royal
new_man = test_emb[0] - test_emb2[4]

new_words3 = test_words + ['new_man']
new_emb3 = np.append(test_emb, new_man.reshape(1, -1), axis=0)

dist3 = pairwise.cosine_similarity(new_emb3, new_emb3)

plot_matrix_distance(new_words3, dist3)

In [ ]:
url = 'https://drive.google.com/u/0/uc?id=1KY6avD5I1eI2dxQzMkR3WExwKwRq2g94'
output = 'gloveembedding.pkl'
gdown.download(url, output, quiet=False)

In [ ]:
# instanciamos embeddings de GloVe
# antes liberamos la RAM de los embeddings de fasttext 
# para tener disponible suficiente RAM para hacer operaciones
del(model_fasttext)
model_glove = GloveEmbeddings()
words_glove=[model_glove.embeddings[i][0] for i in range(model_glove.embeddings.shape[0])]

In [ ]:
# cantidad de palabras en GloVe
model_glove.embeddings.shape

In [ ]:
embeddings_matrix_distance(model_glove, words)

Diferentes embeddings capturan diferentes relaciones semánticas y en distinto grado:
- En los embeddings de Glove morning está muy relacionado con los saludos y este comportamiento podría no ser deseado.
- En los embeddings de Fasttext la similitud entre los dos grupos de palabras elegidas no es tan alta.